In [1]:
import jupyter_manim

In [2]:
%%manim InscribedAngle
from manimlib.scene.scene import Scene
from manimlib.imports import *

class DecimalTextNumber(VMobject):
    CONFIG = {
        "num_decimal_places": 2,
        "include_sign": False,
        "group_with_commas": True,
        "digit_to_digit_buff": 0.05,
        "show_ellipsis": False,
        "unit_type": "font", # tex or font
        "unit": None,  # Aligned to bottom unless it starts with "^"
        "unit_custom_position": lambda mob: mob.set_color(GREEN).shift(RIGHT*0.1),
        "include_background_rectangle": False,
        "edge_to_fix": LEFT,
        "unit_config": {
            "font": "Digital-7",
            "stroke_width": 0,
        },
        "number_config": {
            "font": r"Digital-7",
            "stroke_width": 0,
        }
    }

    def __init__(self, number=0, **kwargs):
        super().__init__(**kwargs)
        self.number = number
        self.initial_config = kwargs

        if isinstance(number, complex):
            formatter = self.get_complex_formatter()
        else:
            formatter = self.get_formatter()
        num_string = formatter.format(number)

        rounded_num = np.round(number, self.num_decimal_places)
        if num_string.startswith("-") and rounded_num == 0:
            if self.include_sign:
                num_string = "+" + num_string[1:]
            else:
                num_string = num_string[1:]

        self.add(*[
            Text(char,color=self.color,**self.number_config)
            for char in num_string
        ])

        # Add non-numerical bits
        if self.show_ellipsis:
            self.add(SingleStringTexMobject("\\dots"))

        if num_string.startswith("-"):
            minus = self.submobjects[0]
            minus.next_to(
                self.submobjects[1], LEFT,
                buff=self.digit_to_digit_buff
            )

        self.num_string = num_string

        if self.unit is not None:
            if self.unit_type == "font":
                self.unit_sign = Text(self.unit,**self.unit_config)
            elif self.unit_type == "tex":
                del self.unit_config["font"]
                self.unit_sign = TexMobject(self.unit,**self.unit_config)
            self.add(self.unit_sign)

        self.arrange(
            buff=self.digit_to_digit_buff,
            aligned_edge=DOWN
        )

        # Handle alignment of parts that should be aligned
        # to the bottom
        for i, c in enumerate(num_string):
            if c == "-" and len(num_string) > i + 1:
                self[i].align_to(self[i + 1], UP)
                self[i].shift(self[i+1].get_height() * DOWN / 2)
            elif c == ",":
                self[i].shift(self[i].get_height() * DOWN / 2)
        if self.unit and self.unit.startswith("^"):
            self.unit_sign.align_to(self, UP)
        #
        if self.include_background_rectangle:
            self.add_background_rectangle()
            
        self.unit_custom_position(self.unit_sign)
        # if num_string[0] == "-" or num_string[0] == "+":
        #     self[0].set_width(0.2)
        #     self[0].set_color(RED)

    def get_formatter(self, **kwargs):
        config = dict([
            (attr, getattr(self, attr))
            for attr in [
                "include_sign",
                "group_with_commas",
                "num_decimal_places",
            ]
        ])
        config.update(kwargs)
        return "".join([
            "{",
            config.get("field_name", ""),
            ":",
            "+" if config["include_sign"] else "",
            "," if config["group_with_commas"] else "",
            ".", str(config["num_decimal_places"]), "f",
            "}",
        ])

    def get_complex_formatter(self, **kwargs):
        return "".join([
            self.get_formatter(field_name="0.real"),
            self.get_formatter(field_name="0.imag", include_sign=True),
            "i"
        ])

    def set_value(self, number, **config):
        full_config = dict(self.CONFIG)
        full_config.update(self.initial_config)
        full_config.update(config)
        new_decimal = DecimalTextNumber(number, **full_config)
        # Make sure last digit has constant height
        #new_decimal.scale(
        #    self[-1].get_height() / new_decimal[-1].get_height()
        #)
        #"""
        height = new_decimal.get_height()
        yPos = new_decimal.get_center()[1]

        for nr in new_decimal:
            if "." != nr.text :
                nr.scale(height/nr.get_height())
                nr.shift([0,(yPos-nr.get_center()[1]),0])
        max_width = max(*[f.get_width() for f in new_decimal[1:]])
        if new_decimal[0].text == "-" or new_decimal[0].text == "+":
            new_decimal[0].set_width(max_width)
            new_decimal[0].set_color(RED)

        #"""
        new_decimal.move_to(self, self.edge_to_fix)
        new_decimal.match_style(self)
        old_family = self.get_family()
        self.submobjects = new_decimal.submobjects
        for mob in old_family:
            # Dumb hack...due to how scene handles families
            # of animated mobjects
            mob.points[:] = 0
        self.number = number
        # if num_string[0] == "-" or num_string[0] == "+":
        #     self[0].set_width(0.2)
        #     self[0].set_color(RED)
        return self

    def get_value(self):
        return self.number

    def increment_value(self, delta_t=1):
        self.set_value(self.get_value() + delta_t)

class CircleWithAngles(VGroup):
    CONFIG = {
        "inner_line_config": {"color":PURPLE_A},
        "outer_line_config": {"color":TEAL_A},
        "inner_arc_config": {"color":PURPLE_A},
        "outer_arc_config": {"color":TEAL_A},
        "tex_1_config": {"color": TEAL_A},
        "tex_2_config": {"color": PURPLE_A},
    }
    def __init__(self, radius=3, ang1=30, ang2=130, ang3=260, small_radius=0.4, **kwargs):
        digest_config(self, kwargs)
        super().__init__(**kwargs)
        circle = Circle(radius=radius)
        vt_1 = ValueTracker(ang1)
        vt_2 = ValueTracker(ang2)
        vt_3 = ValueTracker(ang3)
        p1 = Dot(circle.point_at_angle(ang1*DEGREES))
        p2 = Dot(circle.point_at_angle(ang2*DEGREES))
        p3 = Dot(circle.point_at_angle(ang3*DEGREES))
        in_lines = VMobject(**self.inner_line_config)
        # ------------- LINES
        out_lines = VMobject(**self.outer_line_config)
        # ------------- ANGLES
        out_arc = self.get_arc_between_lines(small_radius,p1,p2,p3)
        in_arc = self.get_inner_angle(small_radius,p1,p2,p3,circle)
        # ------------- LABELS
        theta_2 = TexMobject("2\\theta",**self.tex_2_config)
        theta_1 = TexMobject("\\theta",**self.tex_1_config)
        # ------------- Equals
        theta_1_val = DecimalTextNumber(0,unit="deg",num_decimal_places=3,**self.tex_1_config)
        theta_2_val = DecimalTextNumber(0,unit="deg",num_decimal_places=3,**self.tex_2_config)
        equal = Text("= 2 * ",font="Digital-7")
        theta_eq = VGroup(theta_1_val, equal, theta_2_val)
        theta_eq_temp = VGroup(theta_1_val, equal, theta_2_val)
        theta_eq.arrange(RIGHT,buff=0.6,aligned_edge=DOWN)
        theta_2_val.shift(LEFT*max(*[f.get_width() for f in theta_2_val])*1)
        rectangle = Rectangle(width=theta_eq.get_width()+0.2,height=theta_eq.get_height()+0.2)
        rectangle.move_to(theta_eq)
        theta_eq.add(rectangle)
        # UPDATERS
        p1.add_updater(lambda mob: mob.move_to(circle.point_at_angle(vt_1.get_value()*DEGREES)))
        p2.add_updater(lambda mob: mob.move_to(circle.point_at_angle(vt_2.get_value()*DEGREES)))
        p3.add_updater(lambda mob: mob.move_to(circle.point_at_angle(vt_3.get_value()*DEGREES)))
        in_lines.add_updater(lambda mob: mob.set_points_as_corners([
            p1.get_center(),circle.get_center(),p2.get_center()
        ]))
        out_lines.add_updater(lambda mob: mob.set_points_as_corners([
            p1.get_center(),p3.get_center(),p2.get_center()
        ]))
        out_arc.add_updater(lambda mob: mob.become(self.get_arc_between_lines(small_radius,p1,p2,p3)))
        in_arc.add_updater(lambda mob: mob.become(self.get_inner_angle(small_radius,p1,p2,p3,circle)))
        theta_1.add_updater(
            lambda mob: mob.move_to(
                p3.get_center()+Line(p3.get_center(),out_arc.point_from_proportion(0.5)).get_vector()*1.7)
        )
        theta_2.add_updater(
            lambda mob: mob.move_to(
                circle.get_center()+Line(circle.get_center(),in_arc.point_from_proportion(0.5)).get_vector()*1.7)
        )
        theta_1_val.add_updater(lambda mob: mob.set_value(self.get_inner_angle(1,p1,p2,p3,circle,False)*180/PI))
        theta_2_val.add_updater(lambda mob: mob.set_value(self.get_arc_between_lines(1,p1,p2,p3,False)*180/PI))
        rectangle.max_width = rectangle.get_width()
        def rect_up(mob):
            line = Line(theta_eq_temp.get_left()+LEFT*0.2,theta_eq_temp.get_right()+RIGHT*0.2)
            if line.get_width() > mob.max_width:
                mob.max_width = line.get_width() 
            mob.set_width(mob.max_width)
            # mob.move_to(line)
            mob.align_to(theta_1_val,LEFT)
            mob.shift(LEFT*0.1)
        rectangle.add_updater(rect_up)
        # ------------- Groups
        dots = VGroup(p1,p2,p3)
        vts = Group(vt_1,vt_2,vt_3)
        self.vts = vts
        self.add(
            circle,dots,
            in_lines,out_lines,
            in_arc,out_arc,
            theta_1,theta_2,
            theta_eq,
        )

    def get_arc_between_lines(self, radius, d1, d2, center,mob=True):
        line1 = Line(center.get_center(),d1.get_center())
        line2 = Line(center.get_center(),d2.get_center())
        h = Line(center.get_center(),center.get_center()+RIGHT)
        angle = angle_between_vectors(line1.get_unit_vector(),line2.get_unit_vector())
        h1 = angle_between_vectors(h.get_unit_vector(),line1.get_unit_vector())
        h2 = angle_between_vectors(h.get_unit_vector(),line2.get_unit_vector())
        if line1.get_angle() <= line2.get_angle():
            start_angle = h1
        else:
            start_angle = h2
        arc = Arc(start_angle, angle,radius=radius,arc_center=center.get_center(),**self.outer_arc_config)
        if mob:
            return arc
        else:
            return angle
    
    def get_inner_angle(self, radius,d1,d2,out_center,in_center,mob=True):
        line1 = Line(out_center.get_center(),d1.get_center())
        line2 = Line(out_center.get_center(),d2.get_center())
        h = Line(out_center.get_center(),out_center.get_center()+RIGHT)
        angle = angle_between_vectors(line1.get_unit_vector(),line2.get_unit_vector())
        v1 = Line(in_center.get_center(),d1.get_center())
        start_angle = angle_between_vectors(h.get_unit_vector(),v1.get_unit_vector())
        arc = Arc(start_angle, angle*2,radius=radius,arc_center=in_center.get_center(),**self.inner_arc_config)
        if mob:
            return arc
        else:
            return angle*2

class InscribedAngle(MovingCameraScene):
    def construct(self):
        circle_grp = CircleWithAngles()
        v1, v2, v3 = circle_grp.vts
        eq = circle_grp[-1]
        circle_grp.to_edge(LEFT,buff=1)
        eq.to_edge(RIGHT,buff=1)
        for mob in circle_grp:
            mob.suspend_updating()
            mob.update()
        self.play(Write(circle_grp))
        for mob in circle_grp:
            mob.resume_updating()
        self.wait()
        self.play(v1.set_value,-10,run_time=3,rate_func=linear)
        self.wait()
        self.play(v2.set_value,225,run_time=5,rate_func=there_and_back)
        self.wait()
        self.play(
            v1.set_value,47,
            v2.set_value,110,
            v3.set_value,335,
            run_time=3,
            rate_func=there_and_back
        )
        self.wait()
        circle_grp.remove(eq)
        self.play(
            FadeOut(eq),
            circle_grp[0].scale,0.64,
            circle_grp[0].move_to,ORIGIN,
            circle_grp[0].to_edge,DOWN,{"buff":0.2}
        )
        self.wait()